In [1]:
import pandas as pd
from keras import callbacks
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
inputs = pd.read_csv("../BreastCancer/entradas_breast.csv")
outputs = pd.read_csv("../BreastCancer/saidas_breast.csv")

In [3]:
class Callback(callbacks.Callback):
    SHOW_NUMBER = 25
    
    counter = 0
    epoch = 0

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch = epoch

    def on_train_batch_end(self, batch, logs=None):
        if self.counter == self.SHOW_NUMBER or self.epoch == 1:
            print('Epoch: ' + str(self.epoch) + ' loss: ' + str(logs['loss']))
            if self.epoch > 1:
                self.counter = 0
        self.counter += 1

In [8]:
def createNetwork(
        optimizer,
        loss,
        kernel_initializer,
        activation,
        neurons
    ):
    classifier = Sequential()
    classifier.add(Dense(
        units=neurons,
        activation=activation,
        kernel_initializer=kernel_initializer,
        input_dim=30
    ))
    # units = (number_of_features + number_of_output_neurons)/2
    
    classifier.add(Dropout(0.2))
    
    classifier.add(Dense(
        units=neurons,
        activation=activation,
        kernel_initializer=kernel_initializer,
    ))

    classifier.add(Dropout(0.2))

    classifier.add(Dense(
        units=1,
        activation='sigmoid'  # critical parameter
    ))

    classifier.compile(
        optimizer=optimizer,
        # optimizer = 'adam'
        loss=loss,
        metrics=['binary_accuracy']
    )

    return classifier


IndentationError: unexpected indent (2649146990.py, line 10)

In [5]:
classifier = KerasClassifier(build_fn=createNetwork)
parameters = {
    'batch_size': [10, 30],
    'epochs': [50, 100],
    'optimizer': ['adam', 'sgd'],
    'loss': ['binary_crossentropy', 'hinge'],
    'kernel_initializer': ['random_uniform', 'normal'],
    'activation': ['relu', 'tanh'],
    'neurons': [16, 8]
}

C:\Users\Greze\AppData\Local\Temp\ipykernel_12788\1825768898.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn=createNetwork)


In [6]:
gridSearch = GridSearchCV(
    estimator=classifier,
    param_grid=parameters,
    scoring='accuracy',
    cv=5    
)

gridSearch = gridSearch.fit(
    inputs,
    outputs,
    verbose=0,
    use_multiprocessing = True
)

4/4 [==============================] - 0s 2ms/step


In [7]:
best_params = gridSearch.best_params_
best_accuracy = gridSearch.best_score_

In [9]:
print(best_accuracy)

0.8946281633286757


In [10]:
print(best_params)

{'activation': 'relu', 'batch_size': 10, 'epochs': 100, 'kernel_initializer': 'normal', 'loss': 'binary_crossentropy', 'neurons': 16, 'optimizer': 'adam'}
